## Setup

In [102]:
import pandas as pd
import numpy as np
import datetime

SILVER_PATH = '/home/lucas-nunes/workspace/Postech/challenges/2_ibov/data/silver'
GOLD_PATH = '/home/lucas-nunes/workspace/Postech/challenges/2_ibov/data/gold'

LIST_COLUMNS_DF_FILTER = ['data', 'ultimo']



In [103]:
df = pd.read_parquet(f'{SILVER_PATH}/silver.parquet')

In [104]:
list_all_index = df['item'].drop_duplicates().to_list()

list_all_index_label = list_all_index[:]
list_all_index_label.remove('ibov')

list_all_index

['prata',
 'us30',
 'ouro',
 'spx',
 'ibov',
 'ndx',
 'petroleo',
 'vix',
 'ibov_futuro',
 'eur_brl',
 'usd_brl',
 'ibrx100',
 'BBSA3',
 'CPLE6',
 'ITUB4',
 'VALE3',
 'PETR3',
 'ELET3',
 'ibrx50',
 'FTXIN9',
 'RENT3',
 'us500',
 'SLCE3',
 'cobre',
 'FLRY3',
 'bitcoin',
 'ferro',
 'MGLU3',
 'STOXX50E',
 'ethereum',
 'CRFB3']

In [105]:
df_all_index = pd.DataFrame()

In [106]:
dict_all_index = {}
for item in list_all_index:
    df_item = df.where(df['item'] == item).dropna(how='all')
    dict_all_index[item] = {
        'value': df_item[LIST_COLUMNS_DF_FILTER],
        'start_date': min(df_item['data'])
    }
    

In [107]:
# df_all_index['data'] = dict_index['ibov'].where(df['ultimo'] > 0).sort_values(by='data').dropna(how='all')['data']
# df_all_index['data'] = dict_index['ouro']['value'].where(df['data'] > datetime.date(1994, 1, 1)).dropna(how='all').sort_values(by='data')['data']

list_all_available_days = []
current_date = datetime.date(1994, 1, 1) #  dict_all_index['ibov']['start_date']
last_day = max(dict_all_index['ibov']['value']['data']) # datetime.datetime.now().date()

while current_date != last_day + datetime.timedelta(days=1):
    list_all_available_days.append(current_date)
    current_date += datetime.timedelta(days=1)


In [108]:
df_all_index['data'] = list_all_available_days

for index, df_index in dict_all_index.items():
    df_all_index = df_all_index.merge(how='left', right=dict_all_index[index]['value']).rename(columns={'ultimo': index})

In [109]:
index = 'ibov'
dict_all_index[index]

{'value':               data    ultimo
 2010    1982-01-07       0.0
 2015    1982-01-08       0.0
 2020    1982-01-11       0.0
 2025    1982-01-12       0.0
 2030    1982-01-13       0.0
 ...            ...       ...
 204974  2025-02-10  125572.0
 205004  2025-02-11  126522.0
 205034  2025-02-12  124380.0
 205064  2025-02-13  124850.0
 205093  2025-02-14  128219.0
 
 [10641 rows x 2 columns],
 'start_date': datetime.date(1982, 1, 7)}

In [110]:
def handle_missing_data(row, index, dict_last_value):
    current_value = row[index]
    if row.isnull().any():
        return dict_last_value['value']
    else:
        dict_last_value['value'] = current_value
        return current_value
    

index = 'ibov'

dict_index_processed = {}

for index in list_all_index:

    df_current_index = df_all_index[['data', index]]

    dict_last_value = {'value': 0}

    df_current_index = df_current_index.where(df_current_index['data'] > dict_all_index[index]['start_date']).dropna(how='all')
    df_current_index = df_current_index.where(df_current_index['data'] < datetime.date(2025, 2, 16)).dropna(how='all')
    df_current_index = df_current_index.sort_values(by='data', ascending=True)

    df_current_index[index] = df_current_index.T.apply(lambda x: handle_missing_data(x, index, dict_last_value))

    dict_index_processed[index] = df_current_index


In [111]:
df_all_index_processed = pd.DataFrame()

df_all_index_processed['data'] = list_all_available_days

for index, df_index in dict_index_processed.items():
    df_all_index_processed = df_all_index_processed.merge(how='left', right=df_index)

## Write

In [113]:
df_all_index_processed.to_parquet(f'{GOLD_PATH}/gold.parquet')